# Integration of ddel cassette in the _Saccharomyces cerevisiae_ HIS3 locus

The plasmid pPS1 was used as PCR template to integrate the ddel cassette in the HIS3 locus. The HIS3 locus was chosen since this is also the location of the del construct in RS112 and derivatives.

The resulting strain should be G418R his3 LEU2 and HygS. After recombination in the ddel cassete, the cells will be G418S his3 eu2 and HygR. The cassette will be amplified in such a way that the 2µ and URA3 sequences were not included.

In [1]:
from pygenome import saccharomyces_cerevisiae as sg

In [2]:
from pydna.all import *

In [3]:
HIS3_locus = Dseqrecord( sg.stdgenes["HIS3"].locus() )

The HIS3_locus sequence contain the whole HIS3 locus including promoter and terminator.

In [4]:
HIS3_locus.write("HIS3_locus.gb")

HIS3_locus.gb

Two primers Fw_TEF_His3 and Rv_TEF_His3 were designed for the integration of the cassette into the HIS3 locus. The other primers below were used to confirm the integrity and location of the plasmid or to produce overlapping segments of the cassette.

In [5]:
(Fw_TEF_His3, 
 Rv_TEF_His3,
 A_His3,
 D_His3,
 Fw_FlankingTEFp5,
 Rv_His3d3,
 Rv_flankingHis3d3, 
 p196,
 p195,
 dda11_Kl_tr_f,
 dda12_Kl_tr_r ) = parse('''

>Fw_TEF-His3
AATATACTAAAAAATGAGCAGGCAAGATAAACGAAGGCAAAGATGtcacccggccagcgacatgg
>Rv_TEF-His3
TGTATATATATCGTATGCTGCAGCTTTAAATAATCGGTGTCACTAggcggcgttagtatcgaatcgacag

>A-His3
TGACGACTTTTTCTTAATTCTCGTT 
>D-His3
GCTCAGTTCAGCCATAATATGAAAT

>Fw_FlankingTEFp5
actcttggcctcctctagtacactc
>Rv_His3d3
gggcctctttaaaagcttgaccg
>Rv_flankingHis3d3
cttgcatgtcaagatatatgggacg

>196_pMEC_MX4_fwd (19-mer)
acttttcggggaaatgtgc
>195_pMEC_MX4_rev (20-mer)
ataataccgcgccacatagc

>dda11_Kl_tr_f
CGATGAGTTTTTCTAAGTTTAACTTGATACTACTAGATTT
>dda12_Kl_tr_r
GGCCAGTGAATTCGAGCTCGGTACCCTTAGTATTAGTAAATTTGTTGACA
''')


In [6]:
pPS1 = read("pPS1.gb")

The cassette was amplified as three overlapping PCR products since PCR of the entire cassette gave low yields. 

In [7]:
first_fragment  = pcr(Fw_TEF_His3, dda12_Kl_tr_r, pPS1)
second_fragment = pcr(dda11_Kl_tr_f, p195, pPS1)
third_fragment  = pcr(p196, Rv_TEF_His3 , pPS1)

(first_fragment, second_fragment, third_fragment)

(Amplicon(2694), Amplicon(2884), Amplicon(3247))

In [8]:
first_fragment.name   = "first_fragment"
second_fragment.name  = "second_fragment"
third_fragment.name   = "third_fragment"

The integration of the cassette is simulated below. The recipient strains were RS112 and CEN.PK 

In [9]:
asm = Assembly((HIS3_locus, first_fragment, second_fragment, third_fragment, HIS3_locus))

In [10]:
asm

Assembly
fragments..: 2663bp 2694bp 2884bp 3247bp 2663bp
limit(bp)..: 25
G.nodes....: 10
algorithm..: common_sub_strings

In [11]:
asm.assemble_linear()[0].figure()

BK006948|45
         \/
         /\
         45|first_fragment|442
                           \/
                           /\
                           442|second_fragment|361
                                               \/
                                               /\
                                               361|third_fragment|45
                                                                  \/
                                                                  /\
                                                                  45|BK006948

The figure shows that the overlapping sequences recombine by a 442 bp sequence and a 361 bp sequence to form the cassette which integrates by two 45 bp sequences.

In [12]:
cassette_integrated_in_HIS3_locus = asm.assemble_linear()[0]

In [13]:
cassette_integrated_in_HIS3_locus.write("cassette_integrated_in_HIS3_locus.gb")

cassette_integrated_in_HIS3_locus.gb

In [14]:
cassette_integrated_in_HIS3_locus.seq

Dseq(-9938)
TATT..CTCC
ATAA..GAGG

The cassette integrated in the HIS3 locus is 9938 bp long. A DNA double stranded break somewhere between the repeated sequences can stimulate recombination so that the hygromycing marker is reacticated. Below we use the XbaI restriction enzyme to simulate this break. The enzyme cuts inside the cassette once in the YIplac128 derived sequence.

In [15]:
from Bio.Restriction import XbaI

In [16]:
first_fragment, second_fragment = cassette_integrated_in_HIS3_locus.cut(XbaI)

In [17]:
first_fragment, second_fragment

(Dseqrecord(-7919), Dseqrecord(-2023))

In [18]:
first_fragment.name  = "first_fragment"
second_fragment.name = "second_fragment"

In [19]:
asm = Assembly((first_fragment, second_fragment))

In [20]:
asm.assemble_linear()[1].figure()

first_fragment|400
               \/
               /\
               400|second_fragment

The fragments recombine between two repeated sequences that are 400 bp long.

In [21]:
recombined_cassette = asm.assemble_linear()[1]

In [22]:
recombined_cassette.seq

Dseq(-3620)
TATT..CTCC
ATAA..GAGG

The recombined cassette is 3620 bp long. The cassette should contain the hygromycing resistance open reading frame. We read hyg_cassette variable from a local file. 

In [23]:
hyg_cassette = read("hyg_cassette.gb")

In [24]:
hyg_cassette.list_features()

| Ft# | Label or Note    | Dir | Sta  | End  |  Len | type         | orf? |
|-----|------------------|-----|------|------|------|--------------|------|
|   0 | L:TEF-PRO        | --> | 0    | 368  |  368 | misc_feature |  no  |
|   1 | L:hygromycin B p | --> | 368  | 1397 | 1029 | CDS          | yes  |
|   2 | L:TEF-TER        | --> | 1397 | 1637 |  240 | misc_feature |  no  |

The feature #1 is the hygromycin orf. 

In [25]:
hyg_orf = hyg_cassette.extract_feature(1)

In [26]:
hyg_orf

Dseqrecord(-1029)

The hygromycin orf is present in the recombined cassette.

In [27]:
str(hyg_orf.seq) in recombined_cassette

True

From this analysis follows that the region where a double stranded break can expected to induce cassette recombination is the size of the integrated cassette (9938 bp) minus the size of the cassette after recombination (3620 bp) which is 6318 bp. 